In [1]:
from da_od.config import test_img
from da_od.model import DepthAnythingEstimator, MiDaSEstimator

image_path = test_img / "img-00019.jpg"

depth_estimator = DepthAnythingEstimator(image_path, encoder="vits")
depth_img_colored, depth_raw = depth_estimator.process_image()

xFormers not available
xFormers not available


In [2]:
DepthAnything_estimator = DepthAnythingEstimator(image_path, encoder="vits")
DepthAnything_colored, DepthAnything_raw = DepthAnything_estimator.process_image()

In [3]:
MiDaS_estimator = MiDaSEstimator(image_path, model_type="DPT_Large")
MiDaS_colored, MiDaS_raw = MiDaS_estimator.process_image()

Using cache found in /Users/maryam/.cache/torch/hub/intel-isl_MiDaS_master
